In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

Importing the libraries used in the program

In [ ]:
t1=pd.read_csv("train.csv")
t2=pd.read_csv("test.csv")

Reading the train and test files using the read function of the pandas   

In [ ]:
t1.head()

Printing the first five rows of the train dataset 

In [ ]:
t2.head()

Printing the first five rows of the test dataset 

In [ ]:
t1.shape,t2.shape

Checking the shape of the train and test dataset

In [ ]:
t1.info()
#df[column].astype(str)

In [ ]:
t2.info()

In [ ]:
t1.isnull().sum()

Checking the null values in the columns of the train dataset

In [ ]:
np.dtype(t1["rez_esc"])

In [ ]:
for i in t1.columns:
    if np.dtype(t1[i]) in ["float64","int64"]:
        t1[i]=t1[i].fillna(value=t1[i].median())
    if np.dtype(t1[i]) in ["object"]:
        t1[i]=t1[i].fillna(value=t1[i].mode().iloc[0])

Removing the null values from the train dataset

In [ ]:
t1.isnull().sum()

Checking the null values

In [ ]:
t2.isnull().sum()

The output variable or the target variable is "Target"

Checking the null values in the test dataset

In [ ]:
for i in t2.columns:
    if np.dtype(t2[i]) in ["float64","int64"]:
        t2[i]=t2[i].fillna(value=t2[i].median())
    if np.dtype(t1[i]) in ["object"]:
        t2[i]=t2[i].fillna(value=t2[i].mode().iloc[0])

Removing the null values from the test dataset

In [ ]:
t2.isnull().sum()

In [ ]:
t1.describe().T

Checking the minimum and maximum values using the describe function

In [ ]:
import matplotlib.pyplot as plt
for i in t1.columns:
    if np.dtype(t1[i]) in ["float64","int64"]: 
        plt.figure()
        t1[i].plot(kind='box',figsize=(7,6))

Making the boxplot for the different columns in the train dataset.In the boxplot of all the columns the value is greater then 0 then we need not to remove the outliers.

In [ ]:
labels = t1["Target"]
print(np.unique(labels.values))
fig, ax = plot.subplots()
(labels.value_counts()).plot(ax=ax, kind='bar')

Making the bar graph for the "Target" column in the train dataset that will represent the different bars of the unique values in the target column and also provide the count for the number of unique values in the train dataset

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in t1.columns.drop(["dependency","Target"]):
    t1[i]=le.fit_transform(t1[i])
t1.head()    

Then the label encoding is performed in the train dataset that will encode each of the column present in the dataset. 

In [ ]:
t1["dependency"]=t1["dependency"].replace(to_replace="yes",value=1)
t1["dependency"]=t1["dependency"].replace(to_replace="no",value=0)
#t1["dependency"]=t1["dependency"].astype('int64')

In this we have replaced the value of yes with 1 and the value of no withe 0 

In [ ]:
t1.info()

From the info() we have checked that all the columns are now converted in the int64 datatype

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
f=t1.columns.drop("Target")
s_values=scaler.fit_transform(t1[f])
a=0
for i in f:
    t1[i]=s_values[:,a]
    a+=1

Now we are using the MinMaxScaler function to normalize all the columns present in the train dataset leaving the target column.

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=20)
features=t1.columns.drop("Target")
X=pca.fit_transform(t1[features])
X.shape
pca.explained_variance_ratio_

In this we are using the PCA technique for the feature engineering that will provide us the important columns that have high variance as compared to the remaining columns present in the dataset.In this we are selecting 10 columns from the data

In [ ]:
pc1=PCA(0.95)
pc1.fit_transform(t1[features]).shape


This is used to check that how many columns we have to take that will provideus the 95% variation for the dataset 

In [ ]:
q=pd.DataFrame(X,columns=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t'])
q['Target']=t1['Target']
q.head()

Here q dataframe is created which has the 10 columns that have the high variance which we get from the PCA technique and also a target column

In [ ]:
t1.columns.values

This columns() function returns the name of the columns present in the dataset

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(q,test_size=0.10,random_state=100)
features = train.columns.drop("Target")
target = ["Target"]

In this train_test_split function is used to split the train dataset into two parts train and test.The train part will be used to train the model and test part will be used to test or validate the performance of the model.90% data is given for tarining and 10% data is given for testing.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=15,min_samples_split=25,min_impurity_decrease=0.15)
model.fit(train[features],train[target])
train_accuracy = model.score(train[features],train[target])
test_accuracy = model.score(test[features],test[target])
print(train_accuracy,test_accuracy)

In this the random forest classifier model has been prepared and then the model is trained using the dataset which is separated for the training purpose and then the accuracy of the model hs been tested using the dataset which is seperated for the testing purpose.

Cross Validation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=5,shuffle=False,random_state=100)
features = train.columns.drop("Target")
target = ["Target"]
knn_k_vals = [1,3,5,7,9,11]
avg_train_accuracy = []
avg_val_accuracy = []
X = train[features]
y = train[target]
for k in knn_k_vals: 
    train_accuracy = []
    val_accuracy = []
    for i,(train,val) in enumerate(k_fold.split(X,y)): 
        model = KNeighborsClassifier(n_neighbors=k,metric="euclidean")
        model.fit(X.iloc[train],y.iloc[train])
        train_accuracy.append(model.score(X.iloc[train],y.iloc[train]))
        val_accuracy.append(model.score(X.iloc[val],y.iloc[val]))
    avg_train_accuracy.append(np.mean(train_accuracy))
    avg_val_accuracy.append(np.mean(val_accuracy))

In [ ]:
performance_scores = pd.DataFrame(np.array([knn_k_vals,avg_train_accuracy,avg_val_accuracy]).T,columns=["k","avg_train_accuracy","avg_val_accuracy"])
performance_scores

In [ ]:
best_k = performance_scores["k"][performance_scores["avg_val_accuracy"]==
                                 performance_scores["avg_val_accuracy"].max()]
best_k

With the help of this cross validation technique we will get the value of the n_neighours that at which value of n_neighbour the model is more accurate

In [ ]:
model = KNeighborsClassifier(n_neighbors = 9, metric="euclidean")
model.fit(X,y)
train_accuracy = model.score(X,y)
test_accuracy = model.score(test[features],test[target])
print(train_accuracy,test_accuracy)